In [3]:
import json
with open("hotpot_dev_distractor_v1.json") as f:
    datasets = json.load(f)

In [4]:
datasets = datasets[:500]

In [2]:
from dotenv import load_dotenv
from openai import OpenAI
import os

load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

def llm_create(prompt):
        messages = [{"role": "user", "content": prompt}]
        completion = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages,
        )
        return completion.choices[0].message.content

In [1]:
answer_prompt = """Base on the following document and answer the question.
Only answer, no explanation.

Question:{question}

Document:{docs}
"""

In [5]:
from long_memory.component import WeaviateLongMemory

long_mem = WeaviateLongMemory(user="Hotpot", time_sort=False)

Detect existed Hotpot user group memory space, loading...
Detect existed Hotpot user child memory space, loading...


In [6]:
import pandas as pd

df = pd.DataFrame()
df['question_id'] = None
df['question'] = None
df['gold_answer'] = None
df['answer'] = None
df['org_res_from_long_mem'] = None

In [1]:
import pandas as pd
df = pd.read_json("hotpot_df_v2.json", lines=True)

In [7]:
other_instruct = """Here's some guide for you.
similar_snippets is original memory, related_summaries and closest_summary are compressed memory for suggestion, 
If you see some relative content from related_summaries, you can use jump to related_summaries and get original memory.
The question may need several memory to answer, when you get part of evidence, you can reasoning and use retry to search the rest."""

In [10]:
error_list = []
for i in range(100):
    try:
        print(f"---{i}---")
        long_mem.del_memory()
        doc_list = datasets[i]["context"]
        question_id = datasets[i]["_id"]
        gold_answer = datasets[i]['answer']
        question = datasets[i]["question"]
        df.loc[i, 'question_id'] = question_id 
        df.loc[i, 'question'] = question
        df.loc[i, 'gold_answer'] = gold_answer
        for doc in doc_list:
            article = []
            for content in doc[1]:
                content = content.replace('"', "").strip(".'")
                if len(content)<=7 or content=="<ref name=Ahmed Rashid/The Telegraph> </ref>": # 過濾無意義的詞
                    pass
                else:
                    article.append({"text":content})
            long_mem.add_article(article)
        res = long_mem.get_memory(question, recall=True, other_instruct=other_instruct)
        df.loc[i, 'org_res_from_long_mem'] = str(res)
        answer_res = llm_create(answer_prompt.format(question=question, docs=res))
        df.loc[i, 'answer'] = answer_res
    except Exception as e:
        print(f"-----error{e}-----")
        error_list.append(i)
        
    if (i+1) % 10 == 0:
        df.to_json("hotpot_df_v2.json", orient="records", lines=True)
df.to_json("hotpot_df_v2.json", orient="records", lines=True)

---1---
Detect empty group memory, create memory space...
Detect empty child memory, create memory space...
Save article to long memory done.
Save article to long memory done.
Save article to long memory done.
Save article to long memory done.
Save article to long memory done.
Save article to long memory done.
Save article to long memory done.
Save article to long memory done.
Save article to long memory done.
Save article to long memory done.
---31---
Detect empty group memory, create memory space...
Detect empty child memory, create memory space...
Save article to long memory done.
Save article to long memory done.
Save article to long memory done.
Save article to long memory done.
Save article to long memory done.
Save article to long memory done.
Save article to long memory done.
Save article to long memory done.
Save article to long memory done.
Save article to long memory done.
---51---
Detect empty group memory, create memory space...
Detect empty child memory, create memory spa

In [19]:
error_list # 檢查錯誤 存檔

[]

In [18]:
df.to_json("hotpot_df_v2.json", orient="records", lines=True)

In [22]:
df[:10]

,question_id,question,gold_answer,answer,org_res_from_long_mem
0,5a8b57f25542995d1e6f1371,Were Scott Derrickson and Ed Wood of the same ...,yes,Yes.,"{'search times': 1, 'used queries': ['Were Sco..."
1,5a8c7595554299585d9e36b6,What government position was held by the woman...,Chief of Protocol,Shirley Temple held the position of United Sta...,"{'search times': 3, 'used queries': ['What gov..."
2,5a85ea095542994775f606a8,"What science fantasy young adult series, told ...",Animorphs,Animorphs series,"{'search times': 1, 'used queries': ['What sci..."
3,5adbf0a255429947ff17385a,Are the Laleli Mosque and Esma Sultan Mansion ...,no,No.,"{'search times': 2, 'used queries': ['Are the ..."
4,5a8e3ea95542995a26add48d,"The director of the romantic comedy ""Big Stone...","Greenwich Village, New York City","Greenwich Village, New York City.","{'search times': 2, 'used queries': ['The dire..."
5,5abd94525542992ac4f382d2,2014 S/S is the debut album of a South Korean ...,YG Entertainment,YG Entertainment,"{'search times': 1, 'used queries': ['2014 S/S..."
6,5a85b2d95542997b5ce40028,Who was known by his stage name Aladin and hel...,Eenasul Fateh,Eenasul Fateh,"{'search times': 1, 'used queries': ['Who was ..."
7,5a87ab905542996e4f3088c1,The arena where the Lewiston Maineiacs played ...,"3,677 seated","4,000","{'search times': 1, 'used queries': ['The aren..."
8,5a7bbb64554299042af8f7cc,"Who is older, Annie Morton or Terry Richardson?",Terry Richardson,Terry Richardson is older than Annie Morton.,"{'search times': 2, 'used queries': ['Who is o..."
9,5a8db19d5542994ba4e3dd00,Are Local H and For Against both from the Unit...,yes,Yes.,"{'search times': 1, 'used queries': ['Are Loca..."


In [24]:
eval(df['org_res_from_long_mem'][7])

{'search times': 1,
 'used queries': ['The arena where the Lewiston Maineiacs played their home games can seat how many people?'],
 'searched memory': [{'text': 'The Androscoggin Bank Colisée is a multi-purpose arena in Lewiston, Maine, with a capacity of 4,000, which opened in 1958. Notably, it hosted the World Heavyweight Title fight in 1965, where a famous photograph of Muhammed Ali over Sonny Liston was captured.',
   'time': '2025/01/14 11:30'}],
 'thought': 'The Androscoggin Bank Colisée, where the Lewiston Maineiacs played their home games, has a seating capacity of 4,000.',
 'evidence': [{'text': 'The Androscoggin Bank Colisée is a multi-purpose arena in Lewiston, Maine, with a capacity of 4,000.'},
  {'text': "The Lewiston Maineiacs' home games were held at the Androscoggin Bank Colisée."}]}